# Step1: Dataset Creation

- Sorces : 
  1. https://www.delftstack.com/howto/python/opencv-bounding-box/
  2. https://learnopencv.com/cropping-an-image-using-opencv/#:~:text=There%20is%20no%20specific%20function,And%20it's%20done!
  3. https://stackoverflow.com/questions/26009861/how-can-i-access-the-ordering-of-contours-in-opencv
  4. Grouping : https://stackoverflow.com/questions/64879673/how-to-split-python-list-into-lists-of-different-lengths

In [1]:
# http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghcircles/py_houghcircles.html
import cv2
print(cv2.__version__)
import numpy as np
import warnings
warnings.filterwarnings("ignore")

4.6.0


In [2]:
# load the image
img = cv2.imread('/home/user/Jupyter-Projects/3.Herogliphi_project/Material/GlyphDataset/Dataset/Pictures/egyptianTexts22.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [3]:
original = img.copy()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#image bluing
#gray = cv2.blur(gray,(6,6))
gray = cv2.GaussianBlur(gray, (7, 7), 0)


thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

ROI_number = 0
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]


# Drawing boundary boxes around contours.
ROI_number=0
objectsProb_list=[]
for c in cnts:
    area = cv2.contourArea(c)
    if area > 400 and area <3000:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), None, 0)
        #ROI = img[y:y+h, x:x+w]
        #print("X : ",x,"| Y : ",y)
        objectsProb_list.append([x,y,w,h])
#print(objectsProb_list)


################################################################################################################


# Sorting the objects by x coordinates.
objectsProb_list.sort(key = lambda x: x[0])

objNum_byEachLine=[]
final_list=[]
for d in range(0,12):
    addition = 97 *d        # Number of pixels in each line of writings
    for i in objectsProb_list:
        if i[0] in range(addition,addition+97 ):
            objNum_byEachLine.append(i)
    final_list.append(len(objNum_byEachLine))
    
final_list = [final_list[i + 1] - final_list[i] for i in range(len(final_list)-1)]
final_list.insert(0, 13)
print("Num of objects per each Line is : ",final_list)
print("Num of all objects detected : " , sum(final_list))

myNum = tuple(final_list)
prev = 0
newList = []
for i in myNum:
    newList.append((objectsProb_list[prev:prev+i]))
    prev = prev+i 

print(newList[0])
################################################################################################################
print(img.shape)


# Saving final symbol images.
for obj in objectsProb_list:
    ROI = img[y:y+h, x:x+w]
    
    # choose specific width and height
    if (ROI.shape[0] <= 100)and (ROI.shape[1] <= 90):
        
        # Save final cropped symbols
        #cv2.imwrite('./Generated_data/symbol{}.png'.format(ROI_number), ROI)
        ROI_number += 1
    


img = cv2.resize(img, (960, 900))                # Resize image
#cv2.imshow('image', img)
#cv2.imshow('Thresh',thresh)
cv2.waitKey()
cv2.destroyAllWindows()

Num of objects per each Line is :  [13, 30, 26, 26, 28, 24, 24, 28, 23, 31, 28, 31]
Num of all objects detected :  312
[[8, 603, 29, 165], [23, 898, 72, 77], [23, 463, 52, 41], [28, 621, 59, 66], [30, 830, 70, 64], [31, 217, 62, 69], [33, 1274, 49, 16], [33, 726, 52, 40], [34, 116, 36, 69], [35, 947, 71, 209], [39, 3, 48, 51], [47, 1309, 18, 76], [54, 512, 16, 81]]
(1600, 1160, 3)


In [4]:
# expected number of x pixels for each line
# 1160//12 == 96.7

In [5]:
# real number of object near to this number (Num of contours)
len(objectsProb_list)

312